### Building and training the model

Let's start with importing all indispensable libraries.

In [ ]:
!pip install hazm
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import device
from tqdm import tqdm_notebook
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torchtext
from sklearn.model_selection import train_test_split
import hazm
from sklearn.metrics import confusion_matrix
from tensorboardX import SummaryWriter

Now, we are going to load the tarining and validation sets, but we will use only the clean_review column and label column.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/sentipers/final_sentipers_binary.csv', encoding='utf-8')

# tokenize data
def tokenize(comment):
    return hazm.word_tokenize(comment)


# creating vocab
min_freq = 5
data['tokens'] = data['comment'].apply(lambda t: tokenize(t))
special_tokens = ['<unk>', '<pad>']
vocab = torchtext.vocab.build_vocab_from_iterator(data['tokens'],
                                                  min_freq=min_freq,
                                                  specials=special_tokens)
unk_index = vocab['<unk>']
pad_index = vocab['<pad>']
vocab.set_default_index(unk_index)

# making input ids
# comments are in length between 3 and 256, so we do zero padding for those which have a length less than 256
def numeralize(tokens):
  ids = [vocab[token] for token in tokens]
  ids = np.pad(ids, (0, 256 - len(ids)), 'constant')
  return ids

# making label ids
def toId(label):
  return 1 if label=='positive' else 0


data['ids']=data['tokens'].apply(lambda t: numeralize(t))
data['label']=data['label_id'].apply(lambda t: toId(t))
data['length']=data['ids'].apply(lambda t: len(t))

data = data[['ids','label','length']]


new_data = []
for [ids,label,length] in data.values:
    new_data.append({'ids':torch.tensor(ids),'label':torch.tensor(label),'length':torch.tensor(length)})

In [ ]:
class BiGRU(nn.Module):
    def __init__(self, hidden_size, vocab_size, embedding_dim, output_size, n_layers=1, dropout=0.2,
                 spatial_dropout=True, bidirectional=True):
        
        # Inherit everything from the nn.Module
        super(BiGRU, self).__init__()
        
        # Initialize attributes
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout_p = dropout
        self.spatial_dropout = spatial_dropout
        self.bidirectional = bidirectional
        self.n_directions = 2 if self.bidirectional else 1
        
        # Initialize layers
        self.embedding = nn.Embedding(self.vocab_size, self.embedding_dim)
        self.dropout = nn.Dropout(self.dropout_p)
        if self.spatial_dropout:
            self.spatial_dropout1d = nn.Dropout2d(self.dropout_p)
        self.gru = nn.GRU(self.embedding_dim, self.hidden_size, num_layers=self.n_layers, 
                          dropout=(0 if n_layers == 1 else self.dropout_p), batch_first=True,
                          bidirectional=self.bidirectional)
        # Linear layer input size is equal to hidden_size * 3, becuase
        # we will concatenate max_pooling ,avg_pooling and last hidden state
        self.linear = nn.Linear(self.hidden_size * 3, self.output_size)

        
    def forward(self, input_seq, input_lengths, hidden=None):
        # Extract batch_size
        self.batch_size = input_seq.size(0)
        
        # Embeddings shapes
        # Input: (batch_size,  seq_length)
        # Output: (batch_size, seq_length, embedding_dim)
        emb_out = self.embedding(input_seq)

        if self.spatial_dropout:
            # Convert to (batch_size, embedding_dim, seq_length)
            emb_out = emb_out.permute(0, 2, 1)
            emb_out = self.spatial_dropout1d(emb_out)
            # Convert back to (batch_size, seq_length, embedding_dim)
            emb_out = emb_out.permute(0, 2, 1)
        else:
            emb_out = self.dropout(emb_out)
        
        # Pack padded batch of sequences for RNN module
        packed_emb = nn.utils.rnn.pack_padded_sequence(emb_out, input_lengths, batch_first=True,enforce_sorted=False)
                
        # GRU input/output shapes, if batch_first=True
        # Input: (batch_size, seq_len, embedding_dim)
        # Output: (batch_size, seq_len, hidden_size*num_directions)
        gru_out, hidden = self.gru(packed_emb, hidden)
        # gru_out: tensor containing the output features h_t from the last layer of the GRU
        # gru_out comprises all the hidden states in the last layer ("last" depth-wise, not time-wise)
        # For biGRu gru_out is the concatenation of a forward GRU representation and a backward GRU representation
        # hidden (h_n) comprises the hidden states after the last timestep
        
        # Extract and sum last hidden state
        # Input hidden shape: (n_layers x num_directions, batch_size, hidden_size)
        # Separate hidden state layers
        hidden = hidden.view(self.n_layers, self.n_directions, self.batch_size, self.hidden_size)
        last_hidden = hidden[-1]
        # last hidden shape (num_directions, batch_size, hidden_size)
        # Sum the last hidden state of forward and backward layer
        last_hidden = torch.sum(last_hidden, dim=0)
        # Summed last hidden shape (batch_size, hidden_size)
        
        # Pad a packed batch
        # gru_out output shape: (batch_size, seq_len, hidden_size*num_directions)
        gru_out, lengths = nn.utils.rnn.pad_packed_sequence(gru_out, batch_first=True)
              
        # Sum the gru_out along the num_directions
        if self.bidirectional:
            gru_out = gru_out[:,:,:self.hidden_size] + gru_out[:,:,self.hidden_size:]
        
        # Output dimensions: (batch_size, hidden_size)
        max_pool = F.adaptive_max_pool1d(gru_out.permute(0,2,1), (1,)).view(self.batch_size,-1)
        
        # Output shape: (batch_size, hidden_size)
        avg_pool = torch.sum(gru_out, dim=1) / lengths.view(-1,1).type(torch.FloatTensor) 

        # Concatenate max_pooling, avg_pooling and last hidden state tensors
        concat_out = torch.cat([last_hidden, max_pool, avg_pool], dim=1)

        #concat_out = self.dropout(concat_out)
        out = self.linear(concat_out)
        return F.log_softmax(out, dim=-1)
    
    
    def add_loss_fn(self, loss_fn):
        self.loss_fn = loss_fn
        

    def add_optimizer(self, optimizer):
        self.optimizer = optimizer
        
        
    def add_device(self, device=torch.device('cuda')):
        self.device = device
    
    
    def train_model(self, train_iterator):
        self.train()
        
        train_losses = []
        losses = []
        losses_list = []
        num_seq = 0
        batch_correct = 0
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        for i, batches in tqdm_notebook(enumerate(train_iterator, 1), total=len(train_iterator), desc='Training'):
            input_seq, target, x_lengths = batches['ids'].to(device), batches['label'].to(device), batches['length'].to(device)
            
            self.optimizer.zero_grad()

            pred = self.forward(input_seq, x_lengths)
            loss = self.loss_fn(pred, target)
            loss.backward()
            losses.append(loss.data.cpu().numpy())
            self.optimizer.step()
            
            losses_list.append(loss.data.cpu().numpy())
            
            pred = torch.argmax(pred, 1)

            if self.device.type == 'cpu':
                batch_correct += (pred.cpu() == target.cpu()).sum().item()

            else:
                batch_correct += (pred == target).sum().item()

            num_seq += len(input_seq)     
    
            if i % 100 == 0:
                avg_train_loss = np.mean(losses)
                train_losses.append(avg_train_loss)
                
                accuracy = batch_correct / num_seq
                
                print('Iteration: {}. Average training loss: {:.4f}. Accuracy: {:.3f}'\
                      .format(i, avg_train_loss, accuracy))
                
                losses = []
                
            avg_loss = np.mean(losses_list)
            accuracy = batch_correct / num_seq
                              
        return train_losses, avg_loss, accuracy
    
    
    def evaluate_model(self, eval_iterator, conf_mtx=False):
        self.eval()
        
        eval_losses = []
        losses = []
        losses_list = []
        num_seq = 0
        batch_correct = 0
        pred_total = torch.LongTensor()
        target_total = torch.LongTensor()
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        with torch.no_grad():
            for i, batches in tqdm_notebook(enumerate(eval_iterator, 1), total=len(eval_iterator), desc='Evaluation'):
                input_seq, target, x_lengths = batches['ids'].to(device), batches['label'].to(device), batches['length'].to(device)
                
                pred = self.forward(input_seq, x_lengths)
                loss = self.loss_fn(pred, target)
                losses.append(loss.data.cpu().numpy())
                losses_list.append(loss.data.cpu().numpy())
                
                pred = torch.argmax(pred, 1)
                                
                if self.device.type == 'cpu':
                    batch_correct += (pred.cpu() == target.cpu()).sum().item()
                    
                else:
                    batch_correct += (pred == target).sum().item()
                    
                num_seq += len(input_seq)     
                
                pred_total = torch.cat([pred_total, pred], dim=0)
                target_total = torch.cat([target_total, target], dim=0)
                
                if i % 100 == 0:
                    avg_batch_eval_loss = np.mean(losses)
                    eval_losses.append(avg_batch_eval_loss)
                    
                    accuracy = batch_correct / num_seq
                    
                    print('Iteration: {}. Average evaluation loss: {:.4f}. Accuracy: {:.2f}'\
                          .format(i, avg_batch_eval_loss, accuracy))

                    losses = []
                    
            avg_loss_list = []
                    
            avg_loss = np.mean(losses_list)
            accuracy = batch_correct / num_seq
            
            conf_matrix = confusion_matrix(target_total.view(-1), pred_total.view(-1))
        
        if conf_mtx:
            print('\tConfusion matrix: ', conf_matrix)
            
        return eval_losses, avg_loss, accuracy, conf_matrix


In [ ]:
def collate(batch, pad_index):
    batch_ids = [i['ids'] for i in batch]
    batch_ids = nn.utils.rnn.pad_sequence(batch_ids, padding_value=pad_index, batch_first=True)
    batch_length = [i['length'] for i in batch]
    batch_length = torch.stack(batch_length)
    batch_label = [i['label'] for i in batch]
    batch_label = torch.stack(batch_label)
    batch = {'ids': batch_ids,
             'length': batch_length,
             'label': batch_label}
    return batch

In [ ]:
import functools

batch_size = 16
collate = functools.partial(collate, pad_index=pad_index)

Now we will instantiate the model, add loss function, optimizer, and device to it and begin the training.

In [ ]:
from sklearn.model_selection import KFold

k=5
splits=KFold(n_splits=k,shuffle=True,random_state=42)
foldperf={}

# Initialize parameters
hidden_size = 8
vocab_size = len(vocab)
embedding_dim = 300
output_size = 2
learning_rate = 0.001
n_layers = 1
dropout = 0.5
epochs = 10
spatial_dropout = True

# Check whether system supports CUDA
CUDA = torch.cuda.is_available()
# Move the model to GPU if possible
# if CUDA:
#     model.cuda()

# fasttext
vectors = torchtext.vocab.FastText(language='fa')
pretrained_embedding = vectors.get_vecs_by_tokens(vocab.get_itos())

device = torch.device('cpu')



# Instantiate the EarlyStopping
# early_stop = EarlyStopping(wait_epochs=1)

train_losses_list, train_avg_loss_list, train_accuracy_list = [], [], []
eval_avg_loss_list, eval_accuracy_list, conf_matrix_list = [], [], []


history = {'train_losses': [], 'test_losses': [],'train_accs':[],'test_accs':[]}


for fold, (train_idx,val_idx) in enumerate(splits.split(np.arange(len(data)))):
    train_sampler = torch.utils.data.SubsetRandomSampler(train_idx)
    test_sampler = torch.utils.data.SubsetRandomSampler(val_idx)
    train_dataloader=torch.utils.data.DataLoader(new_data, batch_size=16,collate_fn=collate, sampler=train_sampler)
    test_dataloader=torch.utils.data.DataLoader(new_data, batch_size=16,collate_fn=collate, sampler=test_sampler)
        
    model = BiGRU(hidden_size, vocab_size, embedding_dim, output_size, n_layers, dropout,
              spatial_dropout, bidirectional=False)
    
    model.embedding.weight.data = pretrained_embedding
    model.add_loss_fn(nn.NLLLoss())

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    model.add_optimizer(optimizer)

    model = model.to(device)
    model.add_device(device)

    for epoch in range(epochs):
        
        print('\nStart epoch [{}/{}]'.format(epoch+1, epochs))
        
        train_losses, train_avg_loss, train_accuracy = model.train_model(train_dataloader)
        
        train_losses_list.append(train_losses)
        train_avg_loss_list.append(train_avg_loss)
        train_accuracy_list.append(train_accuracy)
        
        _, eval_avg_loss, eval_accuracy, conf_matrix = model.evaluate_model(test_dataloader)
        
        eval_avg_loss_list.append(eval_avg_loss)
        eval_accuracy_list.append(eval_accuracy)
        conf_matrix_list.append(conf_matrix)

        history['train_losses'].append(train_avg_loss)
        history['test_losses'].append(eval_avg_loss)
        history['train_accs'].append(train_accuracy)
        history['test_accs'].append(eval_accuracy)
        
        print('\nEpoch [{}/{}]: Train accuracy: {:.3f}. Train loss: {:.4f}. Evaluation accuracy: {:.3f}. Evaluation loss: {:.4f}'\
              .format(epoch+1, epochs, train_accuracy, train_avg_loss, eval_accuracy, eval_avg_loss))

    

In [ ]:
def average(numList):
  return sum(numList)/len(numList)

print('accuracy average: ',average(history['test_accs']))
print('loss average: ',average(history['test_losses']))